## Table des matières

- [Identification des données à nettoyer](#nettoyage)
- [Script de nettoyage](#script)

In [1]:
import numpy as np
import pandas as pd

customers= pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
transactions = pd.read_csv('transactions.csv')

In [2]:
customers.head(3)

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984


In [3]:
products.head(3)

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0


In [4]:
transactions.head(3)

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270


<a id="nettoyage"></a>
### <span style="color: #030aa4">Identification des données à nettoyer</span>

In [5]:
def customers_check (df):
    df = customers
    print("Pour la table 'customers':")
    print(" ")
    print(df.dtypes)
    print(" ")
    if df.birth.unique().min() > 1910 :
        print ("Les clients de 'customers' sont tous vivants.")
    if df.birth.unique().min() <= 2004 :
        print ("Les clients de 'customers' sont tous majeurs.")
    if df.duplicated(subset='client_id').sum() == 0 :
        print ("Il n'y a pas de doublons.")
    if list(df.sex.unique()) == ['f','m'] :
        print ("Les clients de 'customers' sont bien soient des hommes soient des femmes.")
    if list(df.isnull().any().unique()) == [False]:
        print ("Il n'y a pas de valeurs nulles.")


def products_check(df):
    df = products
    print("Pour la table 'products':")
    print(" ")
    print(df.dtypes)
    print(" ")
    if list(df.categ.unique())== [0, 1, 2]:
        print("Les produits sont tous de catégorie 0, 1 ou 2.")
    if df.price.min()<0:
        print("Il y a des prix d'une valeur négative.")
    if df.duplicated().sum() == 0 :
        print ("Il n'y a pas de doublons.")
    if list(df.any().unique()) == [True]:
        print ("Il n'y a pas de valeurs manquantes.")

        
def transactions_check(df):
    df = transactions
    print("Pour la table 'transactions':")
    print(" ")
    print(df.dtypes)
    print(" ")
    if set(list(df.id_prod.unique())).issubset(set(list(products.id_prod.unique())))== True:
        print ("Tous les produits dans la table 'transactions' sont enregistrés dans la table 'products'.")
    else :
        print("Il y a des produits dans la table 'transactions' qui ne sont pas enregistrés dans la table 'products'.")
        print("Produits manquants :", set(list(transactions.id_prod.unique()))-set(list(products.id_prod.unique())))
    if set(list(df.client_id.unique())).issubset(set(list(customers.client_id.unique())))== True:
        print ("Tous les clients dans la table 'transactions' sont enregistrés dans la table 'customers'.")
    else :
        print("Il y a des clients dans la table 'transactions' qui ne sont pas enregistrés dans la table 'customers'.")
    if df.duplicated().sum() == 0 :
        print ("Il n'y a pas de doublons.")
    else :
        print("Il y a",df.duplicated().sum(),"doublons.")
    if list(df.any().unique()) == [True]:
        print ("Il n'y a pas de valeurs manquantes.")
    if df['id_prod'].str.contains('T_0').any() == True :
        print("Il y a des valeurs tests à supprimer.")
     
    
customers_check(customers)
print("-"*80)
print(" ")
products_check(products)
print("-"*80)
print(" ")
transactions_check(transactions)

Pour la table 'customers':
 
client_id    object
sex          object
birth         int64
dtype: object
 
Les clients de 'customers' sont tous vivants.
Les clients de 'customers' sont tous majeurs.
Il n'y a pas de doublons.
Les clients de 'customers' sont bien soient des hommes soient des femmes.
Il n'y a pas de valeurs nulles.
--------------------------------------------------------------------------------
 
Pour la table 'products':
 
id_prod     object
price      float64
categ        int64
dtype: object
 
Les produits sont tous de catégorie 0, 1 ou 2.
Il y a des prix d'une valeur négative.
Il n'y a pas de doublons.
Il n'y a pas de valeurs manquantes.
--------------------------------------------------------------------------------
 
Pour la table 'transactions':
 
id_prod       object
date          object
session_id    object
client_id     object
dtype: object
 
Il y a des produits dans la table 'transactions' qui ne sont pas enregistrés dans la table 'products'.
Produits manquants : 

Pour 'transactions' :

- L'analyse sera aussi plus simple à mener si on scinde les informations 'année', 'mois' et 'jour' sur trois colonnes. Pour réaliser cette scission, il faut d'abord convertir les éléments de la colonne 'date' en 'datetime'.
- En l'absence d'une colonne 'quantité', les doublons peuvent être l'achat d'un produit en plusieurs exemplaires lors d'une même session (et donc par le même client et à la même date). Les doublons seront donc conservés.

Pour 'customers' :

Lors de l'analyse, une agrégation par client totalisant leurs dépenses et leur nombre de session a fait apparaître 4  valeurs aberrantes :
    - c_1609 avec 162007,34 euros de dépenses en 12855 sessions
    - c_6714 avec 73220,8 euros de dépenses en 4473 sessions
    - c_3454 avec 54466,38 euros de dépenses en 3275 sessions
    - c_4958 avec 144257,21 euros de dépenses en 2562 sessions
    
Ces valeurs doivent être supprimées dans le script de nettoyage.

<a id="script"></a>
### <span style="color: #030aa4">Script de nettoyage</span>

In [6]:
def cleaning(file_type):
    print(file_type)
       
    if file_type == 'customers' :
        print('entered in customer IF')
        customers_work = pd.read_csv('customers.csv')
        print('done with step 1')
        
        # Suppression de valeurs aberrantes découvertes lors de l'analyse
        customers_work = customers_work[customers_work['client_id']!='c_1609']
        customers_work = customers_work[customers_work['client_id']!='c_6714']
        customers_work = customers_work[customers_work['client_id']!='c_3454']
        customers_work = customers_work[customers_work['client_id']!='c_4958']
        print('done with step 2')
        
        customers_clean = customers_work
        print('done with step 3')
        
        customers_clean.to_csv("customers_clean.csv", index=False)
        print('Done with customers cleaning')
        
    elif file_type == 'products' :
        print('entered in products IF')
        
        products_work = pd.read_csv('products.csv')
        transactions = pd.read_csv('transactions.csv')
        print('done with step 1')
        
        #  Ajout du nouveau produit 0_2245 dans 'products' avec ses attributs :
        #  - 'NewProd' pour son id_prod
        #  - 'NewProdCateg' pour sa catégorie, définie par le préfixe de son id_prod
        #  - 'NewProdPrice' pour son prix, moyenne des prix des produits de même catégorie

        NewProd = list(set(list(transactions.id_prod.unique()))-set(list(products_work.id_prod.unique())))
        
        # Application de la fonction 'first' :
        
        def first(string):
            parts = string.split('_')
            first_part = parts[0] 
            return first_part
        
        NewProdCateg = pd.DataFrame(data= {'categ' : list(set(list(transactions.id_prod.unique()))-set(list(products_work.id_prod.unique())))})
        NewProdCateg = int(NewProdCateg['categ'].apply(first)[0])
        
        NewProdPrice = products_work[products_work.categ== NewProdCateg].price.mean()
        NewProdPrice = round(NewProdPrice,2)
        
        # Création d'un DataFrame avec le nouveau produit :
        
        NouveauProduit = pd.DataFrame(data={'id_prod' : NewProd,
                                            'price' : NewProdPrice,
                                            'categ': NewProdCateg})

        products_work = products_work.append(NouveauProduit,ignore_index=True)
        print('done with step 2')
        
        # Suppression des valeurs négatives :
        products_clean = products_work.drop(products_work[products_work.price<0].index)
        print('done with step 3')
        
        products_clean.to_csv("products_clean.csv", index=False)
        print('Done with products cleaning')
        
    elif file_type == 'transactions' :
        print('entered in transactions IF')
        transactions_work = pd.read_csv('transactions.csv')
        print('done with step 1')
    
        # Suppression des valeurs test avec fonction 'remove_test_values' :
        def remove_test_values(df):
            df = df[df['id_prod'].str.contains('T_0') == False]
            df = df[df['date'].str.contains('test_') == False]
            return df
        transactions_work = remove_test_values(transactions_work)
        print('done with step 2')
        
        # Conversion des valeurs de 'date' en datetime:
        transactions_work['date'] = pd.to_datetime(transactions_work['date'], infer_datetime_format=True)
        print('done with step 3')
    
        # Création des colonnes 'year', 'month', et 'day' depuis la colonne 'date' avec fonction 'extract_date_info'
        def extract_date_info(df):
            df['day'] = df['date'].dt.day 
            df['month'] = df['date'].dt.month
            df['year'] = df['date'].dt.year
        extract_date_info(transactions_work)
        print('done with step 4')
        
        # Suppression de valeurs aberrantes découvertes lors de l'analyse
        transactions_work = transactions_work[transactions_work['client_id']!='c_1609']
        transactions_work = transactions_work[transactions_work['client_id']!='c_6714']
        transactions_work = transactions_work[transactions_work['client_id']!='c_3454']
        transactions_work = transactions_work[transactions_work['client_id']!='c_4958']
        print('done with step 5')
        
        transactions_clean = transactions_work
        print('done with step 6')
        
        transactions_clean.to_csv("transactions_clean.csv", index=False)
        print('Done with transactions cleaning')
        
    else:
        print('Did not go anywhere')
        print("Input is supposed to be either 'customers', 'products', or 'transactions'")

                
cleaning('customers')
print("-"*20)
print(" ")
cleaning('products')
print("-"*20)
print(" ")
cleaning('transactions')

customers
entered in customer IF
done with step 1
done with step 2
done with step 3
Done with customers cleaning
--------------------
 
products
entered in products IF
done with step 1
done with step 2
done with step 3
Done with products cleaning
--------------------
 
transactions
entered in transactions IF
done with step 1
done with step 2
done with step 3
done with step 4
done with step 5
done with step 6
Done with transactions cleaning
